In [51]:
import pandas as pd
import numpy as np
import os
import torch

In [ ]:
gfp_df = pd.read_csv("data/gfp/gfp_dataset.csv")
gfp_df["label"] = gfp_df["inactive"].astype(int)


sub_df = gfp_df[gfp_df.num_muts <= 10]

print("Reducing number of rows from %d to %d" % (len(gfp_df), len(sub_df)))

sub_df.to_csv("data/gfp_dataset_10mut.csv", index=False)
gfp_df = sub_df


Reducing number of rows from 613866 to 523369


In [ ]:
import subprocess

def run_python_script(args, cwd=None):
    """
    Executes a Python script with the given arguments list in the specified directory.
    Prints STDOUT in real time.
    The first element should be the script path, followed by its arguments.
    Example: ["other_script.py", "--foo", "bar"]
    """
    cmd = ["python"] + args
    print("Running command:", " ".join(cmd))
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        cwd=cwd
    )

    while True:
        stdout_line = process.stdout.readline()
        if stdout_line:
            print(stdout_line, end="", flush=True)
        elif process.poll() is not None:
            break

    for line in process.stdout:
        print(line, end="", flush=True)

    stderr = process.stderr.read()
    if stderr:
        print("\n--- STDERR ---\n", stderr)

    process.stdout.close()
    process.stderr.close()
    return process


args = ["../code/embedding_engine.py",
        "--action", "embed_parallel",
        "--dataset_file", "%s/data/gfp/gfp_dataset_10mut.csv" % os.getcwd(),
        #"--n_workers", "5",
        #"--execute_after_parallel_generation", "false",
        "--model", "esm2_t33_650M_UR50D",
        "--evaluation_path", "/%s/results/gfp_embeddings/esm_650m" % os.getcwd(),
        "--sequence_colname", "full_seq",
        "--label_column_name", "label",
        "--job_executing_type", "lsf",
        "--positions_to_embed", "42", "44", "61", "65", "68", "69", "72", "94", "108", "112", "121", "145", "148", "150", "167", "181", "185", "203", "205", "220", "222", "224"]


run_python_script(args, cwd=os.path.abspath("../code"))


Running command: python ../code/embedding_engine.py --action embed_parallel --dataset_file /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/data/gfp_dataset_10mut.csv --model esm2_t33_650M_UR50D --evaluation_path //home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m --sequence_colname full_seq --label_column_name label --job_executing_type lsf --positions_to_embed 42 44 61 65 68 69 72 94 108 112 121 145 148 150 167 181 185 203 205 220 222 224
[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning
[INFO] Successfully loaded '/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/data/gfp_dataset_10mut.csv' with column 'full_seq'.
[INFO] Using evaluation_path: //home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m
[INFO] Tokenizing sequences, this may take a while
[INFO] Caching 
	(1) es

<Popen: returncode: 0 args: ['python', '../code/embedding_engine.py', '--act...>

In [88]:
# Execute workers after jobs parallel generation

jobs_path = "/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel"
args = ["../code/embedding_engine.py",
        "--action", "execute_workers",
        "--job_executing_type", "lsf",
        "--jobs_path", "%s" % jobs_path,
        "--n_workers", "15"]

run_python_script(args, cwd=os.path.abspath("../code"))


# In case something fails!
# args = ["../code/embedding_engine.py",
#         "--action", "execute_workers",
#         "--job_executing_type", "lsf",
#         "--jobs_path", ##### 
#         "--specific_job_i", "1046",
#         "--n_workers", "10"]

# run_python_script(args, cwd=os.path.abspath("../code")) 

Running command: python ../code/embedding_engine.py --action execute_workers --job_executing_type lsf --jobs_path /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel --n_workers 15
[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning
[INFO] Created job execution library at: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel/job_execution_20251211_212118
Submitting job 1 via:  bsub -n 2 -gpu num=1:gmem=24G:aff=yes -R same[gpumodel] -R rusage[mem=64GB] -R span[ptile=2] -e /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel/job_execution_20251211_212118/job_outputs/err_file_worker_53738345 -o /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddi

<Popen: returncode: 0 args: ['python', '../code/embedding_engine.py', '--act...>

In [109]:
args = ["../code/embedding_engine.py",
        "--action", "check_on_jobs",
        "--jobs_path", "%s" % jobs_path]

run_python_script(args, cwd=os.path.abspath("../code"))

Running command: python ../code/embedding_engine.py --action check_on_jobs --jobs_path /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel
[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning
[INFO] Checking on jobs in: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/gfp_embeddings/esm_650m/jobs_20251211_205625_parallel
[INFO] Args: {'average_embeddings': False, 'positions_to_embed': [42, 44, 61, 65, 68, 69, 72, 94, 108, 112, 121, 145, 148, 150, 167, 181, 185, 203, 205, 220, 222, 224], 'dataset_file': '/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/data/gfp_dataset_10mut.csv', 'n_workers': 5, 'print_done_jobs': False, 'execute_after_parallel_generation': False, 'model': 'esm2_t33_650M_UR50D', 'evaluation_path': '//home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/r

<Popen: returncode: 0 args: ['python', '../code/embedding_engine.py', '--act...>

In [ ]:
df = pd.read_csv("data/gfp/gfp_dataset_10mut.csv")
df["sub_indices"] = 0

for i in range(1,11):
    sub_indices_of_nmut = np.where(df["num_muts"] == i)[0]
    df["sub_indices"].iloc[sub_indices_of_nmut] = np.arange(0,len(sub_indices_of_nmut))
    sub_df = df.iloc[sub_indices_of_nmut]
    sub_df.to_csv("data/gfp/gfp_dataset_10mut_nmut_%d.csv" % i, index=False)

df.to_csv("data/gfp/gfp_dataset_10mut.csv", index=False)

/tmp/ipykernel_3576768/3057819653.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["sub_indices"].iloc[sub_indices_of_nmut] = np.arange(0,len(sub_indices_of_nmut))
/tmp/ipykernel_3576768/3057819653.py:6: SettingWithCopyWarning: 
A valu

In [ ]:
base_path = "/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/"
intermediate_result_paths = [#"%s/results/gfp_embeddings/esm_650m/evaluations" % base_path,
                    #"%s/results/gfp_embeddings/esm_35m/evaluations" % base_path,
                    "%s/results/gfp_embeddings/esm_8m/evaluations" % base_path]

final_save_path = [#"%s/results/gfp_embeddings/esm_650m" % base_path,
                #"%s/results/gfp_embeddings/esm_35m" % base_path,
                "%s/data/gfp/embeddings/esm_8m" % base_path]


df = pd.read_csv("data/gfp_dataset_10mut.csv")
for intermediate_result_path, final_save_path in zip(intermediate_result_paths, final_save_path):

    print("Processing %s and saving to %s" % (intermediate_result_path, final_save_path))

    subfolders = [f.name for f in os.scandir(intermediate_result_path) if f.is_dir()]

    for i in range(1, 4):
        final_embeddings = None
        final_labels = None
        final_indices = None

        sub_df = pd.read_csv("%s/data/gfp_dataset_10mut_nmut_%d.csv" % (base_path, i))
        
        sequences_analysed = 0 

        for subfolder in subfolders:
            embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "embeddings.pt"))
            labels = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "y_value.pt"))
            indices = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "indices.pt"))

            if final_embeddings is None:
                final_embeddings = torch.zeros([sub_df.shape[0], embeddings.shape[1], embeddings.shape[2]])
                final_labels = torch.zeros([sub_df.shape[0]])
                final_indices = torch.zeros([sub_df.shape[0]], dtype=torch.int64)

            relevant_indices_wthin_intermediate_result = df.iloc[indices]["num_muts"] == i
            within_sub_df_indices = df.iloc[indices][relevant_indices_wthin_intermediate_result]["sub_indices"]
            
            assert (sum(relevant_indices_wthin_intermediate_result) == len(within_sub_df_indices))
            
            if len(within_sub_df_indices) == 0:
                continue
            
            sequences_analysed += len(within_sub_df_indices)
            print("Found %d sequences for nmut %d in folder %s" % (len(within_sub_df_indices), i, subfolder))
            print("Analyzed %d/%d sequences" % (sequences_analysed, sub_df.shape[0]))
            

            final_embeddings[within_sub_df_indices.to_numpy()] = embeddings[relevant_indices_wthin_intermediate_result.to_numpy()]
            final_labels[within_sub_df_indices.to_numpy()] = labels[relevant_indices_wthin_intermediate_result.to_numpy()].to(torch.float)
            final_indices[within_sub_df_indices.to_numpy()] = indices[relevant_indices_wthin_intermediate_result.to_numpy()]

            if sequences_analysed == sub_df.shape[0]:
                break

        torch.save(final_embeddings, os.path.join(final_save_path, "embeddings_of_nmut_%d.pt" % i))
        torch.save(final_labels, os.path.join(final_save_path, "y_values_of_nmut_%d.pt" % i))
        torch.save(final_indices, os.path.join(final_save_path, "indices_of_nmut_%d.pt" % i))
        
        print("####### CLEARED NMUTS = %d for %s" % (i, final_save_path))
        assert(sum(sub_df["label"] == final_labels.to(torch.int).numpy()) == sub_df.shape[0])
        assert(sum(final_indices == np.where(df["num_muts"] == i)[0]) == sub_df.shape[0])
        print("####### SANITY ASSERTS PASSED")
    break
            


Processing /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//results/gfp_embeddings/esm_8m/evaluations and saving to /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m


/tmp/ipykernel_3576768/2727016933.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "em

Found 21 sequences for nmut 1 in folder 349500_350000
Analyzed 21/56 sequences
Found 3 sequences for nmut 1 in folder 350000_350500
Analyzed 24/56 sequences
Found 1 sequences for nmut 1 in folder 336000_336500
Analyzed 25/56 sequences
Found 1 sequences for nmut 1 in folder 334000_334500
Analyzed 26/56 sequences
Found 1 sequences for nmut 1 in folder 521500_522000
Analyzed 27/56 sequences
Found 1 sequences for nmut 1 in folder 351000_351500
Analyzed 28/56 sequences
Found 1 sequences for nmut 1 in folder 310000_310500
Analyzed 29/56 sequences
Found 1 sequences for nmut 1 in folder 348000_348500
Analyzed 30/56 sequences
Found 5 sequences for nmut 1 in folder 349000_349500
Analyzed 35/56 sequences
Found 1 sequences for nmut 1 in folder 340000_340500
Analyzed 36/56 sequences
Found 1 sequences for nmut 1 in folder 341500_342000
Analyzed 37/56 sequences
Found 1 sequences for nmut 1 in folder 314500_315000
Analyzed 38/56 sequences
Found 1 sequences for nmut 1 in folder 136000_136500
Analyzed 3

/tmp/ipykernel_3576768/2727016933.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "em

Found 5 sequences for nmut 2 in folder 194500_195000
Analyzed 5/711 sequences
Found 1 sequences for nmut 2 in folder 339000_339500
Analyzed 6/711 sequences
Found 1 sequences for nmut 2 in folder 114500_115000
Analyzed 7/711 sequences
Found 1 sequences for nmut 2 in folder 119500_120000
Analyzed 8/711 sequences
Found 7 sequences for nmut 2 in folder 346500_347000
Analyzed 15/711 sequences
Found 1 sequences for nmut 2 in folder 331000_331500
Analyzed 16/711 sequences
Found 1 sequences for nmut 2 in folder 101000_101500
Analyzed 17/711 sequences
Found 86 sequences for nmut 2 in folder 349500_350000
Analyzed 103/711 sequences
Found 34 sequences for nmut 2 in folder 350000_350500
Analyzed 137/711 sequences
Found 9 sequences for nmut 2 in folder 336000_336500
Analyzed 146/711 sequences
Found 1 sequences for nmut 2 in folder 263500_264000
Analyzed 147/711 sequences
Found 1 sequences for nmut 2 in folder 196000_196500
Analyzed 148/711 sequences
Found 1 sequences for nmut 2 in folder 315000_315

/tmp/ipykernel_3576768/2727016933.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "em

Found 1 sequences for nmut 3 in folder 516500_517000
Analyzed 1/3733 sequences
Found 1 sequences for nmut 3 in folder 48000_48500
Analyzed 2/3733 sequences
Found 42 sequences for nmut 3 in folder 194500_195000
Analyzed 44/3733 sequences
Found 13 sequences for nmut 3 in folder 339000_339500
Analyzed 57/3733 sequences
Found 10 sequences for nmut 3 in folder 114500_115000
Analyzed 67/3733 sequences
Found 14 sequences for nmut 3 in folder 119500_120000
Analyzed 81/3733 sequences
Found 1 sequences for nmut 3 in folder 261000_261500
Analyzed 82/3733 sequences
Found 50 sequences for nmut 3 in folder 346500_347000
Analyzed 132/3733 sequences
Found 13 sequences for nmut 3 in folder 331000_331500
Analyzed 145/3733 sequences
Found 1 sequences for nmut 3 in folder 330000_330500
Analyzed 146/3733 sequences
Found 1 sequences for nmut 3 in folder 511000_511500
Analyzed 147/3733 sequences
Found 15 sequences for nmut 3 in folder 101000_101500
Analyzed 162/3733 sequences
Found 169 sequences for nmut 3 i

In [ ]:
import pandas as pd
import numpy as np
import os
import torch

df = pd.read_csv("data/gfp_dataset_10mut.csv")

base_path = "/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/"
intermediate_result_paths = ["%s/results/gfp_embeddings/esm_650m/evaluations" % base_path,
                            "%s/results/gfp_embeddings/esm_35m/evaluations" % base_path,
                            "%s/results/gfp_embeddings/esm_8m/evaluations" % base_path]

final_save_path = ["%s/data/gfp/embeddings/esm_650m" % base_path,
                    "%s/data/gfp/embeddings/esm_35m" % base_path,
                    "%s/data/gfp/embeddings/esm_8m" % base_path]

hidden_dim_size = [#1280, 
480, 320]

df = pd.read_csv("data/gfp_dataset_10mut.csv")

for intermediate_result_path, final_save_path, hs in zip(intermediate_result_paths, final_save_path, hidden_dim_size):

    print("Processing %s and saving to %s" % (intermediate_result_path, final_save_path))

    subfolders = [f.name for f in os.scandir(intermediate_result_path) if f.is_dir()]

    embedding_all = torch.zeros([df.shape[0], 22, hs], dtype=torch.float)
    label_all = torch.zeros([df.shape[0]], dtype=torch.float)
    indices_all = torch.zeros([df.shape[0]], dtype=torch.int64)

    for i, subfolder in enumerate(subfolders):

        print("\tLoading %s [%d/%d]" % (subfolder, i, len(subfolders)))
        embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "embeddings.pt"))
        embeddings = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "embeddings.pt"))
        labels = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "y_value.pt"))
        indices = torch.load(os.path.join(intermediate_result_path, subfolder, "train", "indices.pt"))
        
        indices_all[indices] = indices
        label_all[indices] = labels.to(torch.float)
        embedding_all[indices] = embeddings

    for i in range(1, 11):
        slice_indices = np.where(df["num_muts"] == i)[0]
        print("\tSaving %s [%d/%d]" % (os.path.join(final_save_path, "embeddings_of_nmut_%d.pt" % i), i, 10))
        torch.save(embedding_all[slice_indices], os.path.join(final_save_path, "embeddings_of_nmut_%d.pt" % i))
        print("\tSaving %s [%d/%d]" % (os.path.join(final_save_path, "y_values_of_nmut_%d.pt" % i), i, 10))
        torch.save(label_all[slice_indices], os.path.join(final_save_path, "y_values_of_nmut_%d.pt" % i))
        print("\tSaving %s [%d/%d]" % (os.path.join(final_save_path, "indices_of_nmut_%d.pt" % i), i, 10))
        torch.save(indices_all[slice_indices], os.path.join(final_save_path, "indices_of_nmut_%d.pt" % i))
    
    del embedding_all, label_all, indices_all


In [ ]:
#ASSERTIONS AND VALIDATIONS 
final_save_path = ["%s/data/gfp/embeddings/esm_650m" % base_path,
                    "%s/data/gfp/embeddings/esm_35m" % base_path,
                    "%s/data/gfp/embeddings/esm_8m" % base_path]


for i in range(1, 11):
    sub_df = pd.read_csv("%s/data/gfp_dataset_10mut_nmut_%d.csv" % (base_path, i))

    for final_result_path in final_save_path:
        labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
        indices = torch.load(os.path.join(final_result_path, "indices_of_nmut_%d.pt" % i))


        assert_1 = sum(sub_df["label"].to_numpy() == labels.numpy()) == sub_df.shape[0]
        assert_2 = sum(df.iloc[indices]["label"].to_numpy() == labels.numpy()) == sub_df.shape[0]
        assert_3 = sum(df.iloc[indices]["sub_indices"].to_numpy() == sub_df["sub_indices"].to_numpy()) == sub_df.shape[0]
        assert_4 = sum(np.where(df["num_muts"] == i)[0] == indices.numpy()) == sub_df.shape[0]
        assert_5 = sum(df.iloc[np.where(df["num_muts"] == i)[0]]["label"].to_numpy() == labels.numpy()) == sub_df.shape[0]
        
        assert(assert_1)
        assert(assert_2)
        assert(assert_3)
        assert(assert_4)
        assert(assert_5)

        print("Assert 1: %d, Assert 2: %d, Assert 3: %d, Assert 4: %d, Assert 5: %d" % (assert_1, assert_2, assert_3, assert_4, assert_5))
        print("Sanity assertions passed for %d on %s" % (i, final_result_path))
        
        #assert(sum(labels == df.iloc[indices]["label"]) == len(labels))

/tmp/ipykernel_3576768/2171715667.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
/tm

Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 1 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 1 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 1 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 2 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 2 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1

/tmp/ipykernel_3576768/2171715667.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
/tm

Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 6 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 6 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 6 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m


/tmp/ipykernel_3576768/2171715667.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
/tm

...

Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 8 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 8 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 8 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m


/tmp/ipykernel_3576768/2171715667.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
/tm

Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 9 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 9 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 9 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m


/tmp/ipykernel_3576768/2171715667.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(os.path.join(final_result_path, "y_values_of_nmut_%d.pt" % i))
/tm

Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 10 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_650m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 10 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_35m
Assert 1: 1, Assert 2: 1, Assert 3: 1, Assert 4: 1, Assert 5: 1
Sanity assertions passed for 10 on /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks//data/embeddings/esm_8m


True